# Identify rhythmic genes of datasets without time annotation 

### Tools: CYCLOPS
CYCLOPS reveals human transcriptional rhythms Ron C. Anafi, Lauren J. Francey, John B. Hogenesch, Junhyong Kim Proceedings of the National Academy of Sciences May 2017, 114 (20) 5312-5317; DOI: 10.1073/pnas.1619320114

### Dataset: GTEx Lung (version 8)

### Seed genes: Baboon genes cyclying in more than 20 tissues
Diurnal transcriptome atlas of a primate across major neural and peripheral tissues
BY LUDOVIC S. MURE, HIEP D. LE, GIORGIA BENEGIAMO, MAX W. CHANG, LUIS RIOS, NGALLA JILLANI, MAINA NGOTHO, THOMAS KARIUKI, OURIA DKHISSI-BENYAHYA, HOWARD M. COOPER, SATCHIDANANDA PANDA, SCIENCE16 MAR 2018

## Import packages and modules used in CYCLOPS
Note: Change in addprocs() does not affect the number of procosser used because the function in module scripts are written for 5 processors according to Dr. Anafi's machine. If would like to change the number of processor used in CYCLOPS, edit the module script as well.

In [1]:
# Add processor
addprocs(5)

# Set working directory for each processor
@everywhere basedir = homedir()
@everywhere workdir = string(basedir, "/circa/CYCLOPS/baboon")
@everywhere cd(workdir)

using StatsBase
using MultivariateStats
using Distributions

@everywhere include("CYCLOPS_v6_2a_AutoEncoderModule_multi.jl")
@everywhere include("CYCLOPS_v6_2a_Seed.jl")
@everywhere include("CYCLOPS_v6_2a_PreNPostprocessModule.jl")
@everywhere include("CYCLOPS_v6_2a_CircularStats_U.jl")
@everywhere include("CYCLOPS_v6_2a_MultiCoreModule_Smooth.jl")



using CYCLOPS_v6_2a_AutoEncoderModule_multi
using CYCLOPS_v6_2a_Seed
using CYCLOPS_v6_2a_PreNPostprocessModule
using CYCLOPS_v6_2a_CircularStats_U
using CYCLOPS_v6_2a_MultiCoreModule_Smooth

# datadir = string("/circa/CYCLOPS/baboon")
# seeddir = string("/circa/CYCLOPS/baboon")

## Load datasets and seed gene list
* **Dataset**: Each row represents a gene, and each column represents a sample. 
  * The first row is the header **[1,:]**. 
  * The second column is gene symbol **[:,2]** 
  * The expression data strat from the fourth coulmn **[2:end, 4:end]**. 
* **Seed gene list**: Choose a good seed gene list is quite tricky. According to the suggestion from Dr. Anafi: The seed genes are supposed to reflect a "best guess" at the transcripts that will show circadian oscillations in the tissue being sorted. I do think using the seed list from the human data is a good way to go. 
  * Seed gene list is a two dimentions array with all the seed genes stored at the secondary column **[2:end, 2]**. 

In [2]:
# Load dataset
# cd(datadir)
data = readcsv("gtexExpr_ba9.gct")

# Laod seed gene list
# cd(seeddir)
seed= readcsv("CircaGenes_baboon_edited.txt")
seed_list = seed[2:end,2]

cd(workdir)

## Set parameter
* **Frac_Var**: Set Number of Dimensions of SVD to maintain this fraction of variance
* **DFrac_Var**: Set Number of Dimensions of SVD to so that incremetal fraction of variance of var is at least this much
* **N_trials**: Number of random initial conditions to try for each optimization
* **MaxSeeds**: For cutrank = number of probes(genes) - Maxseeds
* **total_background_num**: Number of background runs for global background refrence statistics
* **n_cores**: Number of machine cores

In [3]:
Frac_Var = 0.85 
DFrac_Var = 0.03 
N_trials =20  
MaxSeeds = 10000
total_background_num=20
n_cores=5

5

## Subset datasets and set seeds

In [4]:
# Extract gene expression
expr = data[2:end, 4:end]
expr = Array{Float64}(expr) 

# Extract gene symbols of gene expression matrix
expr_symbol = data[2:end, 2]

# The number of probes(nprobes) can be used as the total genes input in CYCLOPS
# Because the script was written for microarray data at the first time
nprobes=size(expr)[1]
cutrank=nprobes-MaxSeeds
seed_MinMean=(sort(vec(mean(expr,2))))[cutrank] #mean of dim=2 (mean of rows/genes)

# Reseed the random number generator
srand(123456)

MersenneTwister(UInt32[0x0001e240], Base.dSFMT.DSFMT_state(Int32[-1659974861, 1073470649, 358281436, 1073351076, -836744907, 1073086183, 1497051007, 1072821109, 960204239, 1073516083  …  -761652241, 1073037584, -1988158346, 1073300352, -1977759271, 1347447899, 1840933752, -1237490475, 382, 0]), [1.07189, 1.87502, 1.43586, 1.22449, 1.43677, 1.48514, 1.73852, 1.93363, 1.80625, 1.07452  …  1.23687, 1.88544, 1.67686, 1.97169, 1.10156, 1.98025, 1.00876, 1.10863, 1.78681, 1.48881], 382)

In [5]:
# Get gene symbol and expression data from seed genes
seed_symbol, seed_data = getseed(data,seed_list,.7,.14,seed_MinMean,.975)
seed_data_dispersion = dispersion!(seed_data)
outs, norm_seed_data = GetEigenGenes(seed_data_dispersion,Frac_Var,DFrac_Var,30)

(4, [-0.800641 -0.244913 … 0.0834842 1.20215; -0.0428944 -0.803919 … -0.0157068 -0.327027; -0.621947 -0.179344 … -0.454956 -0.290892; -0.713671 -0.13613 … 0.0843836 -0.38758])

## CYCLOPS ordering

In [6]:
# Predict estimated phases
estimated_phaselist,bestnet,global_var_metrics=CYCLOPS_Order_multicore(outs,norm_seed_data,N_trials)

# Smoothing
global_smooth_metrics = smoothness_measures(seed_data_dispersion, norm_seed_data, estimated_phaselist)
pvals = multicore_backgroundstatistics_global_eigen(seed_data_dispersion, outs, N_trials, total_background_num, global_var_metrics)

	From worker 6:	15000 of max 15000 stochastic epochs
	From worker 2:	15000 of max 15000 stochastic epochs
	From worker 4:	15000 of max 15000 stochastic epochs


Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] uuid4(::MersenneTwister) at ./deprecated.jl:57
 [3] msg_header at /home/pinjouwu9325/.julia/v0.6/IJulia/src/msg.jl:18 [inlined]
 [4] msg_pub(::IJulia.Msg, ::String, ::Dict{String,String}, ::Dict{String,Any}) at /home/pinjouwu9325/.julia/v0.6/IJulia/src/msg.jl:30 (repeats 2 times)
 [5] send_stream(::String) at /home/pinjouwu9325/.julia/v0.6/IJulia/src/stdio.jl:172
 [6] send_stdio(::String) at /home/pinjouwu9325/.julia/v0.6/IJulia/src/stdio.jl:130
 [7] (::Base.##302#303{IJulia.#send_stdout,Timer})() at ./event.jl:436
while loading In[6], in expression starting on line 2


	From worker 3:	15000 of max 15000 stochastic epochs
	From worker 5:	15000 of max 15000 stochastic epochs
	From worker 6:	290 of max 5000.0 non-stochastic epochs
	From worker 6:	103 of max 5000 bold epochs
	From worker 3:	380 of max 5000.0 non-stochastic epochs
	From worker 2:	426 of max 5000.0 non-stochastic epochs
	From worker 5:	400 of max 5000.0 non-stochastic epochs
	From worker 3:	103 of max 5000 bold epochs
	From worker 2:	105 of max 5000 bold epochs
	From worker 5:	102 of max 5000 bold epochs
	From worker 4:	777 of max 5000.0 non-stochastic epochs
	From worker 4:	105 of max 5000 bold epochs
	From worker 6:	0.54024326587232315000 of max 15000 stochastic epochs
	From worker 3:	0.540244725473194715000 of max 15000 stochastic epochs
	From worker 2:	0.542711040522440715000 of max 15000 stochastic epochs
	From worker 5:	0.540241623454913415000 of max 15000 stochastic epochs
	From worker 4:	0.542714193096037515000 of max 15000 stochastic epochs
	From worker 2:	368 of max 5000.0 non-st

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] uuid4(::MersenneTwister) at ./deprecated.jl:57
 [3] msg_header at /home/pinjouwu9325/.julia/v0.6/IJulia/src/msg.jl:18 [inlined]
 [4] msg_pub(::IJulia.Msg, ::String, ::Dict{String,String}, ::Dict{String,Any}) at /home/pinjouwu9325/.julia/v0.6/IJulia/src/msg.jl:30 (repeats 2 times)
 [5] send_stream(::String) at /home/pinjouwu9325/.julia/v0.6/IJulia/src/stdio.jl:172
 [6] send_stdio(::String) at /home/pinjouwu9325/.julia/v0.6/IJulia/src/stdio.jl:130
 [7] (::Base.##302#303{IJulia.#send_stdout,Timer})() at ./event.jl:436
while loading In[6], in expression starting on line 6


	From worker 3:	0.542714894113619415000 of max 15000 stochastic epochs
	From worker 6:	0.540240787912021215000 of max 15000 stochastic epochs
	From worker 2:	227 of max 5000.0 non-stochastic epochs
	From worker 4:	298 of max 5000.0 non-stochastic epochs
	From worker 2:	105 of max 5000 bold epochs
	From worker 4:	105 of max 5000 bold epochs
	From worker 5:	773 of max 5000.0 non-stochastic epochs
	From worker 6:	784 of max 5000.0 non-stochastic epochs
	From worker 3:	807 of max 5000.0 non-stochastic epochs
	From worker 5:	102 of max 5000 bold epochs
	From worker 6:	105 of max 5000 bold epochs
	From worker 3:	107 of max 5000 bold epochs
	From worker 2:	0.554265294494326315000 of max 15000 stochastic epochs
	From worker 4:	0.55655996907875415000 of max 15000 stochastic epochs
	From worker 2:	321 of max 5000.0 non-stochastic epochs
	From worker 2:	105 of max 5000 bold epochs
	From worker 5:	0.544620015646839715000 of max 15000 stochastic epochs
	From worker 4:	475 of max 5000.0 non-stochast

...

	From worker 5:	0.548260020598293515000 of max 15000 stochastic epochs
	From worker 3:	0.538932175801298515000 of max 15000 stochastic epochs
	From worker 2:	668 of max 5000.0 non-stochastic epochs
	From worker 6:	969 of max 5000.0 non-stochastic epochs
	From worker 5:	668 of max 5000.0 non-stochastic epochs
	From worker 4:	0.55759577578849415000 of max 15000 stochastic epochs
	From worker 2:	104 of max 5000 bold epochs
	From worker 6:	110 of max 5000 bold epochs
	From worker 3:	574 of max 5000.0 non-stochastic epochs
	From worker 5:	111 of max 5000 bold epochs
	From worker 3:	104 of max 5000 bold epochs
	From worker 4:	451 of max 5000.0 non-stochastic epochs
	From worker 4:	105 of max 5000 bold epochs
	From worker 5:	0.559013095051657315000 of max 15000 stochastic epochs
	From worker 6:	0.566506392225173815000 of max 15000 stochastic epochs
	From worker 2:	0.540927407699922315000 of max 15000 stochastic epochs
	From worker 6:	106 of max 5000.0 non-stochastic epochs
	From worker 3:	0.5

	From worker 3:	0.553218237626220715000 of max 15000 stochastic epochs
	From worker 2:	498 of max 5000.0 non-stochastic epochs
	From worker 2:	104 of max 5000 bold epochs
	From worker 4:	1185 of max 5000.0 non-stochastic epochs
	From worker 5:	0.558998707014725915000 of max 15000 stochastic epochs
	From worker 4:	104 of max 5000 bold epochs
	From worker 6:	0.56390649494769815000 of max 15000 stochastic epochs
	From worker 3:	662 of max 5000.0 non-stochastic epochs
	From worker 3:	104 of max 5000 bold epochs
	From worker 5:	381 of max 5000.0 non-stochastic epochs
	From worker 2:	0.540929910311165115000 of max 15000 stochastic epochs
	From worker 5:	104 of max 5000 bold epochs
	From worker 4:	0.566015848918303915000 of max 15000 stochastic epochs
	From worker 2:	437 of max 5000.0 non-stochastic epochs
	From worker 3:	0.538929626816520215000 of max 15000 stochastic epochs
	From worker 2:	104 of max 5000 bold epochs
	From worker 5:	0.548257740237071615000 of max 15000 stochastic epochs
	Fr

	From worker 5:	0.544278111114932115000 of max 15000 stochastic epochs
	From worker 3:	0.552132071343651415000 of max 15000 stochastic epochs
	From worker 4:	0.558659677345641315000 of max 15000 stochastic epochs
	From worker 3:	441 of max 5000.0 non-stochastic epochs
	From worker 5:	561 of max 5000.0 non-stochastic epochs
	From worker 3:	101 of max 5000 bold epochs
	From worker 5:	104 of max 5000 bold epochs
	From worker 2:	5000 of max 5000.0 non-stochastic epochs
	From worker 2:	107 of max 5000 bold epochs
	From worker 3:	0.552122991669370515000 of max 15000 stochastic epochs
	From worker 5:	0.544035399796106315000 of max 15000 stochastic epochs
	From worker 4:	1261 of max 5000.0 non-stochastic epochs
	From worker 4:	103 of max 5000 bold epochs
	From worker 5:	243 of max 5000.0 non-stochastic epochs
	From worker 2:	0.558316861986479315000 of max 15000 stochastic epochs
	From worker 5:	103 of max 5000 bold epochs
	From worker 3:	434 of max 5000.0 non-stochastic epochs
	From worker 3:	

	From worker 3:	0.553316314382758915000 of max 15000 stochastic epochs
	From worker 4:	559 of max 5000.0 non-stochastic epochs
	From worker 2:	349 of max 5000.0 non-stochastic epochs
	From worker 4:	103 of max 5000 bold epochs
	From worker 2:	103 of max 5000 bold epochs
	From worker 6:	0.559824422026167115000 of max 15000 stochastic epochs
	From worker 4:	0.557531143273204315000 of max 15000 stochastic epochs
	From worker 2:	0.563360963602968715000 of max 15000 stochastic epochs
	From worker 2:	204 of max 5000.0 non-stochastic epochs
	From worker 2:	105 of max 5000 bold epochs
	From worker 4:	546 of max 5000.0 non-stochastic epochs
	From worker 4:	102 of max 5000 bold epochs
	From worker 6:	1123 of max 5000.0 non-stochastic epochs
	From worker 6:	104 of max 5000 bold epochs
	From worker 2:	0.55607714295429615000 of max 15000 stochastic epochs
	From worker 4:	0.55948109971007215000 of max 15000 stochastic epochs
	From worker 2:	460 of max 5000.0 non-stochastic epochs
	From worker 6:	0.5

3-element Array{Float64,1}:
 0.05
 0.05
 0.05

## Cosinor statistics
**cosinor table:**

1.  ""            
2. "Description" (Gene Symbol)
3. "Name"       (Gene ID)
4. "pval"       
5. "bon_pval"   
6. "phase"      
7. "amp"        
8. "fitmean"    
9. "mean"       
10. "rsq"        
11. "ptr"

In [7]:
# Import Filter_cosinor_Output function for filtering bon_pval, ptr, rsq in the cosinor result
function Filter_Cosinor_Output(cosdata::Array{Any,2},pval,ptr,rsq) 
    significant_data=cosdata[append!([1],1 + findin(cosdata[2:end,5] .< pval,true)),:];
    phys_sig_data=significant_data[append!([1],1 + findin(significant_data[2:end,11] .> ptr,true)),:];
    strong_data=phys_sig_data[append!([1],1 + findin(phys_sig_data[2:end,10].> rsq,true)),:];
    strong_data
end

Filter_Cosinor_Output (generic function with 1 method)

In [8]:
estimated_phaselist = mod.(estimated_phaselist, 2*pi)
cosinor = Compile_MultiCore_Cosinor_Statistics(data, estimated_phaselist, 4, 24)
# Filter bon_pval<0.05, ptr>1.66, rsq>0
sig_cosinor = Filter_Cosinor_Output(cosinor, 0.05, 1.66, 0)

elapsed time: 2.770954719 seconds


Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] uuid4(::MersenneTwister) at ./deprecated.jl:57
 [3] msg_header at /home/pinjouwu9325/.julia/v0.6/IJulia/src/msg.jl:18 [inlined]
 [4] msg_pub(::IJulia.Msg, ::String, ::Dict{String,String}, ::Dict{String,Any}) at /home/pinjouwu9325/.julia/v0.6/IJulia/src/msg.jl:30 (repeats 2 times)
 [5] send_stream(::String) at /home/pinjouwu9325/.julia/v0.6/IJulia/src/stdio.jl:172
 [6] send_stdio(::String) at /home/pinjouwu9325/.julia/v0.6/IJulia/src/stdio.jl:130
 [7] (::Base.##302#303{IJulia.#send_stdout,Timer})() at ./event.jl:436
while loading In[8], in expression starting on line 2


2425×11 Array{Any,2}:
      ""  "Description"    …       "mean"    "rsq"       "ptr" 
   104    "RP4-758J18.13"         4.54284   0.246572    1.9623 
   109    "VWA1"                 16.3147    0.471027    5.36103
   110    "ATAD3C"                2.21003   0.31268     5.25267
   148    "RER1"                 25.267     0.257224    1.90201
   229    "KLHL21"         …     20.3781    0.290149    2.03719
   245    "VAMP3"                43.3568    0.395287    2.97486
   287    "GPR157"                0.480872  0.269076    2.51473
   291    "H6PD"                  5.71485   0.325307    2.2516 
   292    "SPSB1"                 4.92211   0.367226    4.70519
   312    "RBP7"           …      5.74945   0.325993   11.6139 
   324    "DFFA"                 11.9241    0.258403    1.69278
   327    "CASZ1"                 0.522162  0.439353    4.28908
     ⋮                     ⋱                            ⋮      
 55501    "SRPK3"                 2.6826    0.275488    2.28275
 55511    "RENBP" 

## Align all genes with Ebox(Par-BZip) phase
Ebox genes: DBP, HLF, TEF

According to Dr. Anafi, Ebox genes in mouse lung peak on average at CT 11.5

In [9]:
eboxgenes = ["DBP", "HLF", "TEF"]
eboxcosinor = cosinor[findin(cosinor[:,2], eboxgenes), :]

# Criteria(? not sure what is it for
# 4=pval,  9=mean, 11=ptr
# Unable to meet the original mean cutoff(100)
# Because the code is designed for microarray datasets, the mean of expression value are belongs to array expression values
# However, here we are try to analyse GTEx gene expression profile which expression values are TPM, so mean is set to 10 
criteria = ((eboxcosinor[:,4].<0.05) &(eboxcosinor[:,9].>10) & (eboxcosinor[:,11].>1.25)) 
eboxcosinor_fit = eboxcosinor[findin(criteria, true), :]

# Extract phases of Ebox genes
eboxphases = Array{Float64}(eboxcosinor[:,6])
eboxphases_mean = Circular_Mean(eboxphases)
estimated_phaselist_adj = mod.(estimated_phaselist .- eboxphases_mean + (pi), 2*pi)

# Count the number of genes which phase>pi or phase<pi
Nday=length(findin((estimated_phaselist_adj .>pi), true))
Nnight=length(findin((estimated_phaselist_adj .<pi), true))

# Adjust the numebr of the genes belongs to day and night if the number of night gene > the number of day gene
# e.g. 3/2*pi goes to 1/2*pi
if (Nday < Nnight)
    estimated_phaselist_adj = mod.(2*pi - estimated_phaselist_adj, 2*pi)
end

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] &(::BitArray{1}, ::BitArray{1}) at ./deprecated.jl:57
 [3] include_string(::String, ::String) at ./loading.jl:522
 [4] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /home/pinjouwu9325/.julia/v0.6/IJulia/src/execute_request.jl:193
 [5] (::Compat.#inner#6{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJulia.Msg}})() at /home/pinjouwu9325/.julia/v0.6/Compat/src/Compat.jl:125
 [6] eventloop(::ZMQ.Socket) at /home/pinjouwu9325/.julia/v0.6/IJulia/src/eventloop.jl:8
 [7] (::IJulia.##13#16)() at ./task.jl:335
while loading In[9], in expression starting on line 9


209-element Array{Float64,1}:
 3.33116 
 4.15945 
 4.51455 
 2.90547 
 3.97175 
 4.57336 
 5.16575 
 5.3195  
 3.84143 
 5.12178 
 0.277824
 4.72683 
 3.4782  
 ⋮       
 4.09661 
 4.1171  
 4.03326 
 5.24188 
 5.28082 
 5.77903 
 2.93383 
 4.3755  
 4.45068 
 4.43172 
 4.27222 
 4.13033 

## Cosinor statistics after Ebox phase alignment
The input has changed to the adjusted phase after Ebox alignment

In [10]:
cosinor_adj = Compile_MultiCore_Cosinor_Statistics(data, estimated_phaselist_adj, 4, 24)
# Filter bon_pval<0.05, ptr>1.66, rsq>0
sig_cosinor_adj = Filter_Cosinor_Output(cosinor_adj, 0.05, 1.66, 0)

elapsed time: 1.612446978 seconds


2417×11 Array{Any,2}:
      ""  "Description"    …       "mean"    "rsq"       "ptr" 
   104    "RP4-758J18.13"         4.54284   0.246572    1.9623 
   109    "VWA1"                 16.3147    0.471027    5.36103
   110    "ATAD3C"                2.21003   0.31268     5.25267
   148    "RER1"                 25.267     0.257224    1.90201
   229    "KLHL21"         …     20.3781    0.290149    2.03719
   245    "VAMP3"                43.3568    0.395287    2.97486
   287    "GPR157"                0.480872  0.269076    2.51473
   291    "H6PD"                  5.71485   0.325307    2.2516 
   292    "SPSB1"                 4.92211   0.367226    4.70519
   312    "RBP7"           …      5.74945   0.325993   11.6139 
   324    "DFFA"                 11.9241    0.258403    1.69278
   327    "CASZ1"                 0.522162  0.439353    4.28908
     ⋮                     ⋱                            ⋮      
 55501    "SRPK3"          …      2.6826    0.275488    2.28275
 55511    "RENBP" 

## Save output file

In [11]:
# Set output directory
outdir = string(basedir, "/circa/CYCLOPS/baboon")
cd(outdir)
writecsv("Sig_Cosinor_ba9.csv", sig_cosinor_adj)

In [12]:
# Output estimated phases
colnames = ["SAMPID" "Estimated_Phase"]
samplephase = hcat(data[1,4:end], estimated_phaselist_adj)
output = vcat(colnames, samplephase)
writecsv("GTEx_estimatedPhase_ba9.csv", output)